In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import time
import datetime
import numpy as np
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import mean_squared_error
from textblob import TextBlob
from lightgbm import LGBMRegressor

%matplotlib inline

line = pd.read_csv('train.csv')

/Users/tiangewan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
line.drop('price',axis=1).shape

(33538, 64)

In [4]:
dff = pd.read_csv('test.csv')
dff.shape

(17337, 64)

In [5]:
df = pd.concat([line.drop('price',axis=1),dff],ignore_index=True)

In [6]:
len(df[:len(line)]) == len(line)

True

In [7]:
len(df[len(line):])==len(dff)

True

In [8]:
def helper(x):
    try:
        a = int(x[:-1])
    except:
        a = x
    return a
zz = np.mean(df.dropna(subset=['host_since']).host_since.apply(lambda x:time.mktime(datetime.datetime.strptime(str(x), "%Y-%m-%d").timetuple())))
def hh(x):
    try:
        return time.mktime(datetime.datetime.strptime(str(x), "%Y-%m-%d").timetuple())
    except:
        return zz
zr = np.mean(df.dropna(subset=['first_review']).first_review.apply(lambda x:time.mktime(datetime.datetime.strptime(str(x), "%Y-%m-%d").timetuple())))
def hhh(x):
    try:
        return time.mktime(datetime.datetime.strptime(str(x), "%Y-%m-%d").timetuple())
    except:
        return zr
zrr = np.mean(df.dropna(subset=['last_review']).last_review.apply(lambda x:time.mktime(datetime.datetime.strptime(str(x), "%Y-%m-%d").timetuple())))
def hhhh(x):
    try:
        return time.mktime(datetime.datetime.strptime(str(x), "%Y-%m-%d").timetuple())
    except:
        return zrr

,email,facebook,google,government_id,identity_manual,jumio,kba,manual_offline,manual_online,offline_government_id,...,phone,photographer,reviews,selfie,sent_id,sesame,sesame_offline,weibo,work_email,zhima_selfie
0,1,0,0,1,0,1,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,1,0,0,...,1,0,1,0,0,0,0,0,0,0
2,1,1,0,0,0,0,1,0,0,0,...,1,0,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,1,0,0,0,...,1,0,1,0,0,0,0,0,0,0
4,1,0,0,1,0,1,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0


In [ ]:
zl = []
for i in range(len(df)):
    zl+=df['host_verifications'][i][1:-1].replace("'",'').replace(" ",'').split(',')
cv = np.unique(zl)
gl = pd.DataFrame(columns=cv[1:])
for i in tqdm(range(len(df))):
    content = []
    for j in cv[1:]:
        if j in df['host_verifications'][i][1:-1].replace("'",'').replace(" ",'').split(','):
            content.append(1)
        else:
            content.append(0)
    
    gl.loc[i] = content
gl.head()

In [9]:
cl = []
for g in range(len(df)):
    cl+= df['amenities'][0][1:-1].replace('"','').replace(" ",'').split(',')
dd = np.unique(cl)
con = pd.DataFrame(columns=dd)
for i in tqdm(range(len(df))):
    content = []
    for j in dd:
        if j in df['amenities'][i][1:-1].replace("'",'').replace(" ",'').split(','):
            content.append(1)
        else:
            content.append(0)
    con.loc[i] = content
con.head()

In [10]:
X = pd.DataFrame()
X['interaction'] = df.interaction.fillna(0).apply(lambda x:1 if x else 0)
X['host_neighbourhood'] = df.host_neighbourhood.fillna(0).apply(lambda x:1 if x else 0)
X['neighbourhood_cleansed'] = df.neighbourhood_cleansed.fillna(0).apply(lambda x:1 if x else 0)
X['notes'] = df.notes.fillna(0).apply(lambda x:1 if x else 0)
X['access'] = df.access.fillna(0).apply(lambda x:1 if x else 0)
X['host_about'] = df.host_about.fillna(0).apply(lambda x:1 if x else 0)
X['house_rules'] = df.house_rules.fillna(0).apply(lambda x:1 if x else 0)
X['neighborhood_overview'] = df.neighborhood_overview.fillna(0).apply(lambda x:1 if x else 0)
X['space'] = df.space.fillna(0).apply(lambda x:1 if x else 0)
X['summary'] = df.summary.fillna(0).apply(lambda x:1 if x else 0)
X['house_rules'] = df.house_rules.fillna('').apply(lambda x:TextBlob(x).sentiment.polarity)
X['access'] = df.access.fillna('').apply(lambda x:TextBlob(x).sentiment.polarity)
X['transit'] = df.transit.fillna('').apply(lambda x:TextBlob(x).sentiment.polarity)
X['neighborhood_overview'] = df.neighborhood_overview.fillna('').apply(lambda x:TextBlob(x).sentiment.polarity)
X['zipcode'] = df.zipcode.fillna('NO!').apply(lambda x:str(x))
X['maximum_nights'] = df['maximum_nights']
X['host_location'] = df['host_location'].fillna('l')
X['host_since'] = df.host_since.apply(hh)
X['host_response_time'] = df.host_response_time.fillna('Not Available')#oh
X['host_response_rate'] = df.host_response_rate.apply(helper).fillna(np.mean(df.host_response_rate.apply(helper)))
X['host_is_superhost'] = df.host_is_superhost.fillna('N')#oh
X['host_listings_count'] = df.host_listings_count.fillna(1)
X['host_has_profile_pic'] = df.host_has_profile_pic.fillna('Na')#oh
X['host_identity_verified'] = df.host_identity_verified.fillna('Nal')#oh
X['neighbourhood_group_cleansed'] = df.neighbourhood_group_cleansed.fillna('Nah')#oh
X['property_type'] = df.property_type.fillna('i')#oh
X['room_type'] = df.room_type.fillna('i')#oh
X['accommodates'] = df.accommodates.fillna(0)
X['bathrooms'] = df.bathrooms.fillna(0)
X['bedrooms'] = df.bedrooms.fillna(0)
X['beds'] = df.beds.fillna(0)
X['bed_type'] = df.bed_type
X['guests_included'] = df['guests_included']
X['extra_people'] = df['extra_people'].apply(lambda x:float(x[1:]))
X['minimum_nights'] = df['minimum_nights']
X['number_of_reviews'] = df['number_of_reviews']
X['first_review'] = df.first_review.apply(hhh)
X['last_review'] = df.last_review.apply(hhhh)
X['review_scores_rating'] = df['review_scores_rating'].fillna(np.mean(df.dropna(subset=['review_scores_rating']).review_scores_rating))
X['review_scores_accuracy'] = df['review_scores_accuracy'].fillna(np.mean(df.dropna(subset=['review_scores_accuracy']).review_scores_accuracy))
X['review_scores_cleanliness'] = df['review_scores_cleanliness'].fillna(np.mean(df.dropna(subset=['review_scores_cleanliness']).review_scores_cleanliness))
X['review_scores_checkin'] = df['review_scores_checkin'].fillna(np.mean(df.dropna(subset=['review_scores_checkin']).review_scores_checkin))
X['review_scores_communication'] = df['review_scores_communication'].fillna(np.mean(df.dropna(subset=['review_scores_communication']).review_scores_communication))
X['review_scores_location'] = df['review_scores_location'].fillna(np.mean(df.dropna(subset=['review_scores_location']).review_scores_location))
X['review_scores_value'] = df['review_scores_value'].fillna(np.mean(df.dropna(subset=['review_scores_value']).review_scores_value))
X['instant_bookable'] = df.instant_bookable
X['cancellation_policy'] = df.cancellation_policy
X['require_guest_profile_picture'] = df.require_guest_profile_picture
X['require_guest_phone_verification'] = df.require_guest_phone_verification
X['calculated_host_listings_count'] = df.calculated_host_listings_count
X['reviews_per_month'] = df['reviews_per_month'].fillna(np.mean(df.dropna(subset=['reviews_per_month']).reviews_per_month))
for i in gl.columns:
    X[i] = gl[i]

for i in con.columns:
    X[i] = con[i]


In [11]:
numcol = []
for i in X.columns:
    if type(X[i][0])==str:
        numcol.append(i)

In [12]:
nums = Pipeline([('ohe',OneHotEncoder(sparse=False))])
ct = ColumnTransformer([('num',nums,numcol)],remainder = 'passthrough')
XD=pd.DataFrame(ct.fit_transform(X))
y=line['price']

In [13]:
XD[:len(line)].shape

(33538, 2105)

In [14]:
from lightgbm import LGBMRegressor
n_features=XD[:len(line)].shape[1]
model = LGBMRegressor(boosting_type='gbdt', objective='regression', num_leaves=n_features,
                                learning_rate= 0.001, n_estimators=10000, max_depth=13,
                                metric='rmse', bagging_fraction=0.8, feature_fraction=0.8, 
                               reg_lambda=9,random_state=199+30)
model.fit(XD[:len(line)],y)
ans = model.predict(XD[len(line):])
ans

array([418.44649257, 122.94231092,  57.93768391, ...,  90.72931713,
        57.88033381, 124.6180921 ])

In [15]:
rez =pd.DataFrame()
rez['Id'] = dff.id
rez['Predicted'] = ans
rez.to_csv('submission222.csv',index=False)

In [17]:
rez

,Id,Predicted
0,19307997,418.446493
1,20176193,122.942311
2,19485371,57.937684
3,13079990,66.956957
4,22339757,59.816082
5,7717071,160.937153
6,19416433,147.126746
7,28602814,220.226703
8,3167768,291.697484
9,22026277,293.252347
